In [20]:
#forked from https://github.com/fchollet/keras/blob/master/examples/cifar10_cnn.py

from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import scipy.misc
import glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")

batch_size = 25
nb_classes = 2
nb_epoch = 200
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# images are RGB.
img_channels = 3


#Changed the input data 
C1,C2 = [],[]
for image_path in glob.glob("C:/cnn/images/clocks/*.png"):
    image = scipy.misc.imread(image_path)
    image = np.rollaxis(image, 0, 3)
    image = np.rollaxis(image, 0, 3)
    C1.append(image)
    
for image_path in glob.glob("C:/cnn/images/crocodiles/*.png"):
    image = scipy.misc.imread(image_path)
    image = np.rollaxis(image, 0, 3)
    image = np.rollaxis(image, 0, 3)
    C2.append(image)
    
X_train = []
X_train.extend(C1[:400])
X_train.extend(C2[:400])
X_test = []
X_test.extend(C1[400:])
X_test.extend(C2[400:])
y_train=[]
for j in range(2):
    for i in range(400):
        y_train.append([j])
        
y_test=[]
for j in range(2):
    for i in range(100):
        y_test.append([j])
        
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
print('Y_train shape:', y_train.shape)



X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

X_train shape: (800L, 3L, 32L, 32L)
800 train samples
200 test samples
Y_train shape: (800L, 1L)


In [ ]:
# Convert class vectors to binary class matrices.
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, Y_train,
                                     batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))
model.save('C:/cnn/model%i.h5' % int(nb_epoch))

Using real-time data augmentation.
Epoch 1/200
800/800 [==============================] - 2s - loss: 0.7271 - acc: 0.5463 - val_loss: 0.6767 - val_acc: 0.5050

There is not all progress shown, just first 10 epochs. 

In [58]:
#Lets predict new image:
from keras.models import load_model
import cv2

model = load_model('C:/cnn/model200.h5')

def predict_image(path_to_image):
    image = scipy.misc.imread(path_to_image)
    image = cv2.resize(image, (32, 32))
    image = np.rollaxis(image, 0, 3)
    image = np.rollaxis(image, 0, 3)
    image = image.astype('float32')
    image /= 255
    x = []
    x.append(image)
    x = np.array(x)
    pred = model.predict(x)
    if pred[0][0]>0.5:
        print(path_to_image, 'is Clock with probability of',pred[0][0]*100, 'percent')
    else:
        print(path_to_image, 'is Crocodile with probability:',pred[0][1]*100, 'percent')

        
predict_image('C:/cnn/clock.jpg')
predict_image('C:/cnn/crocodile.jpg')

C:/cnn/clock.jpg is Clock with probability of 100.0 percent
C:/cnn/crocodile.jpg is Clock with probability of 99.9995708466 percent
